<a href="https://colab.research.google.com/github/igcampos/Mestrado/blob/main/saturacao_explicit_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
import sys
from collections.abc import Iterable
import timeit

In [2]:
#função para o cálculo do modelo Corey - AJUSTADO

def perm_corey(sw,swi,sor,kro_swi,krw_sor,no,nw):
  kro = []
  krw = []

  if isinstance(sw, Iterable):
    #cálculo da permeabilidade relativa do óleo 
    for i in sw:
        kr = kro_swi*((1-sor-i)/(1-swi-sor))**no
        kro.append(kr)
    kro = np.array([kro_swi if i >= 1 else i for i in kro]) #substituindo o valor 1 por kro_swi
    kro[-1] = 0.0 # solução temporária para o erro 

    #cálculo da permeabilidade relativa da água
    for i in sw:
        kr = krw_sor*((i-swi)/(1-swi-sor))**nw
        if kr > krw_sor:
          kr = krw_sor
        krw.append(kr)
    krw = np.array(krw)


  else:
    #cálculo da permeabilidade relativa do óleo
    kr = kro_swi*((1-sor-sw)/(1-swi-sor))**no
    if kr > kro_swi:
      kr = kro_swi
    kro.append(kr)
    kro = np.array(kro) 
    #cálculo da permeabilidade relativa da água 
    kr = krw_sor*((sw-swi)/(1-swi-sor))**nw
    if kr > krw_sor:
      kr = krw_sor
    krw.append(kr)
    krw = np.array(krw)
  
  return kro, krw

#função que calcula a mobilidade em diferentes pontos de saturação
def mobilidade(sw,visc_agua,visc_oleo):

  numerador_da_funcao = perm_corey(sw,swi,sor,kro_swi,krw_sor,no,nw)[1]/visc_agua
  denominador_da_funcao = (perm_corey(sw,swi,sor,kro_swi,krw_sor,no,nw)[1]/visc_agua) + (perm_corey(sw,swi,sor,kro_swi,krw_sor,no,nw)[0]/visc_oleo)
  resultado_mobilidade =  numerador_da_funcao/denominador_da_funcao

  return resultado_mobilidade

In [3]:
def explicit_saturation():

  #Discretização do domínio 
  
  nx = int(input("Digite o número de pontos do grid: "))                 #Número de pontos do grid
  dominio = np.arange(nx)                                                #vetor domínio
  print("Número de dimensões: {}".format(dominio.ndim))                  #número de dimensões
  print(dominio)

  #Calculo de Delta X

  #tamanho do meio poroso
  tam_meio_poroso = int(input("Digite o tamanho do reservatório em ft:"))
  #tamanho do elemento do grid
  tam_grid = tam_meio_poroso/nx
  #Delta X - normalizado
  dx = tam_grid/tam_meio_poroso
  print("") 
  print("O tamanho dos elementos do grid em uma malha estruturada é {}, e o delta X é {} ".format(tam_grid, dx))
  print("") 

  #Calculo de Delta T

  #viscosidade da água 
  visc_agua = float(input("Digite o valor da viscosidade da água em cp:"))
  #viscosidade do óleo
  visc_oleo = float(input("Digite o valor da viscosidade do óleo em cp:"))
  #velocidade total na injeção 
  ut = float(input("Digite o valor da velocidade total de injeção em ft³/d:"))
  #porosidade
  porosidade_meio = float(input("Entre com a porosidade do meio:"))
  #tempo de simulação 
  nt = int(input("Entre com os dias de injeção que será simulado:"))
  #passo de tempo
  passo_tempo = 10 #dias
  #Delta T - normalizado
  dt =  ut*passo_tempo/(tam_meio_poroso*porosidade_meio) #DÚVIDA: seria o passo de tempo ou o tempo total
  print("") 
  print("O delta T normalizado é {}".format(dt))

  #Número de Courant

  num_Courant = dt/dx
  print("") 
  print(f"O número de courant é {num_Courant}")

  #condição de contorno e inicial

  #Condição Inicial Sw(X,0)
  sw_CI = float(input("Entre com a condição de inicial (Sw):"))
  #Condição de Contorno Sw(0,T)
  sw_CC = float(input("Entre com a condição de contorno (Sw):"))
  #Matriz de saturação para alocação dos resultados
  tempos_de_injecao = int(nt/passo_tempo)   #quantidade representando os tempos de injeção que serão calculados
  matriz_resultado_saturacao = np.zeros((tempos_de_injecao,nx)) 
  print(f'A matriz saturação tem a seguinte forma {matriz_resultado_saturacao.shape}')

  #modificando a primeira linha da matriz para alocar os valores das condições iniciais
  matriz_resultado_saturacao[0,:] = sw_CI 

  #Características do sistema 

  #saturação inicial
  swi = float(input("Entre com a saturação de água inicial (Swi):"))
  #saturação de óleo residual
  sor = float(input("Entre com a saturação de óleo residual (Sor):"))
  #permeabilidade relativa da água na saturação residual
  krw_sor = float(input("Entre com a permeabilidade relativa da água no ponto terminal krw(Sor):"))
  #permeabilidade relativa do óleo na saturação residual
  kro_swi = float(input("Entre com a permeabilidade relativa do óleo no ponto terminal kro(Swi):"))
  #porosidade
  porosidade = porosidade_meio
  #parâmetros de Corey
  no = float(input("Insira o parâmetro de Corey (no):"))
  nw = float(input("Insira o parâmetro de Corey (nw):"))

  tempos_de_injecao = np.arange(tempos_de_injecao) #utilizado como vetor de tempo para o calculo de saturação
  print(f'Vetor com as iterações temporais: {tempos_de_injecao}')

  #Calculo da saturação e preenchimento da matriz de resultado

  #calulo do tempo de execução:
  start = timeit.default_timer()

  for n in tempos_de_injecao:
    #calculo da primeira linha
    if n == 0:
      for i in dominio:
        if i == 0: 
          #primeiro ponto
          mobilidade_i_menos_meio = mobilidade(sw_CC,visc_agua,visc_oleo)
          mobilidade_i = mobilidade(sw_CI,visc_agua,visc_oleo)
          sw_n_mais_1 = dt/dx*(mobilidade_i_menos_meio - mobilidade_i) + sw_CI
          matriz_resultado_saturacao[n,i] =  sw_n_mais_1
        else:
          #outros ponto
          sw_ajustado =  (matriz_resultado_saturacao[n,i-1] + matriz_resultado_saturacao[n,i])/2
          mobilidade_i_menos_meio = mobilidade(sw_ajustado,visc_agua,visc_oleo)
          mobilidade_i = mobilidade(sw_CI,visc_agua,visc_oleo)
          sw_n_mais_1 = dt/dx * (mobilidade_i_menos_meio - mobilidade_i) + sw_CI
          matriz_resultado_saturacao[n,i] =  sw_n_mais_1
    #Cálculo das demais linhas      
    else:
        for i in dominio:
          if i == 0: 
            #primeiro ponto
            mobilidade_i_menos_meio = mobilidade(sw_CC,visc_agua,visc_oleo)
            mobilidade_i = mobilidade(matriz_resultado_saturacao[n-1,i],visc_agua,visc_oleo)
            sw_n_mais_1 = dt/dx*(mobilidade_i_menos_meio - mobilidade_i) + matriz_resultado_saturacao[n-1,i]
            matriz_resultado_saturacao[n,i] =  sw_n_mais_1
          else:
            #outros ponto
            sw_ajustado =  (matriz_resultado_saturacao[n-1,i-1] + matriz_resultado_saturacao[n-1,i])/2
            mobilidade_i_menos_meio = mobilidade(sw_ajustado,visc_agua,visc_oleo)
            mobilidade_i = mobilidade(matriz_resultado_saturacao[n-1,i],visc_agua,visc_oleo)
            sw_n_mais_1 = dt/dx * (mobilidade_i_menos_meio - mobilidade_i) + matriz_resultado_saturacao[n-1,i]
            matriz_resultado_saturacao[n,i] =  sw_n_mais_1
  
  stop = timeit.default_timer()
  print('Tempo de execução: {} segundos'.format(stop-start))

  #plotar os resultados
  vetor_comprimento = np.linspace(0, 1, nx)
  fig = plt.figure(figsize=(8,5))
  ax = plt.subplot(1,1,1)
  ax.plot(vetor_comprimento, matriz_resultado_saturacao[0], label = "0.006 PVI ")
  ax.plot(vetor_comprimento, matriz_resultado_saturacao[6], label = "0.042 PVI")
  ax.plot(vetor_comprimento, matriz_resultado_saturacao[20], label = "0.126 PVI")
  ax.plot(vetor_comprimento, matriz_resultado_saturacao[35], label = "0.216 PVI")
  ax.plot(vetor_comprimento, matriz_resultado_saturacao[71], label = "0.432 PVI")

  ax.set_xlabel("x/L", fontsize=15)
  ax.set_ylabel("Saturação da água", fontsize=15)
  plt.title("Curva de Saturação", fontsize=20)
  ax.legend()
  plt.show()

  return